<a href="https://colab.research.google.com/github/kimseungwoo99/fine-tuning-whisper/blob/main/finetuning_whipser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install evaluate>=0.30
!pip install jiwer
!pip install accelerate -U
!pip install transformers[torch]
!pip install wandb

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-jydhfe3g
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-jydhfe3g
  Resolved https://github.com/huggingface/transformers to commit d751dbecb267e4e3ae77352e97a6470da93a0094
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.35.0.dev0-py3-none-any.whl size=7854578 sha256=1ce8586303c9546f30174d197bb49cccf4b6b91af526abe809737dee137f4fed
  Stored in directory: /tmp/pip-ephem-wheel-cache-tb5t1mkb/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successful

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 27.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=52b637d88d08862ee0834ada4bb07ca9e467577bafc0719e8807a64bc56b891d
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
# 훈련이 끝난 모델을 HuggingFace Hub에 업로드하기 위해 로그인
# 비공개 혹은 제한된 공개의 데이터셋에 접근할 경우에도 로그인이 필요하다.
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import WhisperFeatureExtractor
# 파인튜닝을 진행하고자 하는 모델의 feature extractor를 로드
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")

In [ ]:
from transformers import WhisperTokenizer
# 파인튜닝을 진행하고자 하는 모델의 tokenizer를 로드
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", language="Korean", task="transcribe")

In [ ]:
input_str = "저는 서울중앙지검 지능범죄수사팀 최인호 검사입니다."
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 저는 서울중앙지검 지능범죄수사팀 최인호 검사입니다.
Decoded w/ special:    <|startoftranscript|><|ko|><|transcribe|><|notimestamps|>저는 서울중앙지검 지능범죄수사팀 최인호 검사입니다.<|endoftext|>
Decoded w/out special: 저는 서울중앙지검 지능범죄수사팀 최인호 검사입니다.
Are equal:             True


In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="Korean", task="transcribe")

In [ ]:
from tqdm import tqdm
import pandas as pd

In [ ]:
# csv 파일을 로드하여 df로 변환
df = pd.read_csv("/content/drive/MyDrive/dataframe/df_total.csv", index_col=0)

In [ ]:
# Null data 유무 확인
df.isnull().values.sum()

0

In [ ]:
# interval 열 삭제
df = df.drop('interval', axis=1)

In [ ]:
df.head()

,sentence,audio_file
0,고생한 아이들을 위해 맛있는 저녁을 준비 중이신 할머니,/content/drive/MyDrive/final_dataset/P-004-005...
1,할머니 배고파요 밥 주세요 오야 밥 주께,/content/drive/MyDrive/final_dataset/P-004-005...
2,오오 내가 좋아하는 거 먹어야지 이제는 시키지 않아도,/content/drive/MyDrive/final_dataset/P-004-005...
3,알아서 척척 반찬도 옮기는데요,/content/drive/MyDrive/final_dataset/P-004-005...
4,하루 사이에 달라져도 너무 달라진 아이들,/content/drive/MyDrive/final_dataset/P-004-005...


In [ ]:
import pandas as pd
import re

# file_name 열 제외한 나머지 열 전처리
columns_to_preprocess = df.columns[df.columns != 'audio_file']  # 'Text' 열을 제외한 열 선택

for column in columns_to_preprocess:
    df[column] = df[column].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)).replace('\n', ' ').strip())  # 정규식을 사용하여 전처리 및 공백 제거


# 전처리된 데이터프레임 출력
print(df)


                            sentence  \
0     고생한 아이들을 위해 맛있는 저녁을 준비 중이신 할머니   
1             할머니 배고파요 밥 주세요 오야 밥 주께   
2      오오 내가 좋아하는 거 먹어야지 이제는 시키지 않아도   
3                   알아서 척척 반찬도 옮기는데요   
4             하루 사이에 달라져도 너무 달라진 아이들   
...                              ...   
7104                             무서워   
7105                             무거워   
7106                              저희   
7107                         죽을 뻔했어요   
7108                          발 빠졌어요   

                                             audio_file  
0     /content/drive/MyDrive/final_dataset/P-004-005...  
1     /content/drive/MyDrive/final_dataset/P-004-005...  
2     /content/drive/MyDrive/final_dataset/P-004-005...  
3     /content/drive/MyDrive/final_dataset/P-004-005...  
4     /content/drive/MyDrive/final_dataset/P-004-005...  
...                                                 ...  
7104  /content/drive/MyDrive/final_dataset/P-004-005...  
7105  /content/drive/MyDrive/final_dataset/P-004-005...

In [ ]:
from datasets import Dataset, DatasetDict
from datasets import Audio

In [ ]:
# 오디오 파일 경로를 dict의 "audio" 키의 value로 넣고 이를 데이터셋으로 변환
# 이때, Whisper가 요구하는 사양대로 Sampling rate는 16,000으로 설정한다.
ds = Dataset.from_dict({"audio": [path for path in df["audio_file"]],
                       "Text": [transcript for transcript in df["sentence"]]}).cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
# 데이터셋을 훈련 데이터와 테스트 데이터, 밸리데이션 데이터로 분할
train_testvalid = ds.train_test_split(test_size=0.2)
test_valid = train_testvalid["test"].train_test_split(test_size=0.5)
datasets = DatasetDict({
    "train": train_testvalid["train"],
    "test": test_valid["test"],
    "valid": test_valid["train"]})


In [ ]:
def prepare_dataset(batch):
    # 오디오 파일을 16kHz로 로드
    audio = batch["audio"]

    # input audio array로부터 log-Mel spectrogram 변환
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # target text를 label ids로 변환
    batch["labels"] = tokenizer(batch["Text"]).input_ids
    return batch

In [ ]:
# 데이터 전처리 함수를 데이터셋 전체에 적용
low_call_voices = datasets.map(prepare_dataset, remove_columns=datasets.column_names["train"], num_proc=None)

Map:   0%|          | 0/5687 [00:00<?, ? examples/s]

Map:   0%|          | 0/711 [00:00<?, ? examples/s]

Map:   0%|          | 0/711 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset
low_call_voices = datasets.load_from_disk("/content/drive/MyDrive/hi")

In [ ]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

In [ ]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # 인풋 데이터와 라벨 데이터의 길이가 다르며, 따라서 서로 다른 패딩 방법이 적용되어야 한다. 그러므로 두 데이터를 분리해야 한다.
        # 먼저 오디오 인풋 데이터를 간단히 토치 텐서로 반환하는 작업을 수행한다.
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Tokenize된 레이블 시퀀스를 가져온다.
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # 레이블 시퀀스에 대해 최대 길이만큼 패딩 작업을 실시한다.
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # 패딩 토큰을 -100으로 치환하여 loss 계산 과정에서 무시되도록 한다.
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # 이전 토크나이즈 과정에서 bos 토큰이 추가되었다면 bos 토큰을 잘라낸다.
        # 해당 토큰은 이후 언제든 추가할 수 있다.
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [ ]:
# 훈련시킬 모델의 processor, tokenizer, feature extractor 로드
from transformers import WhisperTokenizer,  WhisperFeatureExtractor
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="Korean", task="transcribe")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", language="Korean", task="transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")

In [ ]:
# 데이터 콜레이터 초기화
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import evaluate

metric = evaluate.load('cer')

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # pad_token을 -100으로 치환
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # metrics 계산 시 special token들을 빼고 계산하도록 설정
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    cer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")

In [ ]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="repo_name",  # 원하는 리포지토리 이름을 임력한다.
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # 배치 크기가 2배 감소할 때마다 2배씩 증가
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,  # epoch 대신 설정
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="cer",  # 한국어의 경우 'wer'보다는 'cer'이 더 적합할 것
    greater_is_better=False,
    push_to_hub=True,
)


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=low_call_voices["train"],
    eval_dataset=low_call_voices["valid"],  # or "test"
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Cer
1000,0.525800,0.833072,0.005037


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


KeyboardInterrupt: ignored

In [ ]:
kwargs = {
    "dataset_tags": "https://huggingface.co/datasets/aoome123/important",
    "dataset": "important",  # a 'pretty' name for the training dataset
    "dataset_args": "config: ko, split: valid",
    "language": "ko",
    "model_name": "ft_model",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-base",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}


In [ ]:
trainer.push_to_hub(**kwargs)

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/290M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

events.out.tfevents.1698340503.33cf5c051a9e.5249.0:   0%|          | 0.00/31.7k [00:00<?, ?B/s]

'https://huggingface.co/aoome123/repo_name/tree/main/'

In [ ]:
processor.push_to_hub("repo_name")
tokenizer.push_to_hub("repo_name")

CommitInfo(commit_url='https://huggingface.co/aoome123/repo_name/commit/90eae73bccb7b6e7408b2b5452e332daa70b9531', commit_message='Upload tokenizer', commit_description='', oid='90eae73bccb7b6e7408b2b5452e332daa70b9531', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# 파인 튜닝한 모델을 로드
from transformers import WhisperForConditionalGeneration, WhisperProcessor, WhisperFeatureExtractor, WhisperTokenizer

fine_model = WhisperForConditionalGeneration.from_pretrained("aoome123/repo_name")


fine_feature_extractor = WhisperFeatureExtractor.from_pretrained("aoome123/repo_name")
fine_tokenizer = WhisperTokenizer.from_pretrained("aoome123/repo_name", language="Korean", task="transcribe")
fine_processor = WhisperProcessor.from_pretrained("aoome123/repo_name")


(…)me123/repo_name/resolve/main/config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/290M [00:00<?, ?B/s]

(…)name/resolve/main/generation_config.json:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

(…)me/resolve/main/preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

(…)_name/resolve/main/tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

(…)ome123/repo_name/resolve/main/merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

(…)3/repo_name/resolve/main/normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

(…)repo_name/resolve/main/added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

(…)ame/resolve/main/special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="repo_name",  # 원하는 리포지토리 이름을 임력한다.
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # 배치 크기가 2배 감소할 때마다 2배씩 증가
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="cer",  # 한국어의 경우 'wer'보다는 'cer'이 더 적합할 것
    greater_is_better=False,
    push_to_hub=False,
)


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=fine_model,
    train_dataset=low_call_voices["train"],
    eval_dataset=low_call_voices["valid"],  # for evaluation(not validation)
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=fine_processor.feature_extractor,  # 수정된 부분
)



In [ ]:
trainer.evaluate() # 기존 whisper-base 모델

{'eval_loss': 3.8098347187042236,
 'eval_cer': 260.7021517553794,
 'eval_runtime': 198.9805,
 'eval_samples_per_second': 3.573,
 'eval_steps_per_second': 0.447}

In [ ]:
trainer.evaluate() # 파인 튜닝된 모델

{'eval_loss': 0.2424846887588501,
 'eval_cer': 11.46092865232163,
 'eval_runtime': 176.3846,
 'eval_samples_per_second': 4.031,
 'eval_steps_per_second': 0.505}

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=fine_processor.feature_extractor,
    model=fine_model,
    padding='longest',
    pad_to_multiple_of=None
)


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=processor.feature_extractor,
    model=model,
    padding='longest',
    pad_to_multiple_of=None
)

In [ ]:
from transformers import DataCollatorForSeq2Seq

small_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
small_processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Korean", task="transcribe")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Korean", task="transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

small_data_collator = DataCollatorForSeq2Seq(
    tokenizer=small_processor.feature_extractor,  # fine_processor.feature_extractor 대신 tokenizer 사용
    model=small_model,  # fine_model 대신 base_model 사용
    padding='longest',
    pad_to_multiple_of=None
)


In [ ]:
# small 모델
from transformers import WhisperTokenizer,  WhisperFeatureExtractor
from transformers import WhisperProcessor
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="repo_name",  # 원하는 리포지토리 이름을 임력한다.
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # 배치 크기가 2배 감소할 때마다 2배씩 증가
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="cer",  # 한국어의 경우 'wer'보다는 'cer'이 더 적합할 것
    greater_is_better=False,
    push_to_hub=False,
)

tiny_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
tiny_trainer = Seq2SeqTrainer(
    model=tiny_model,
    args=training_args,
    eval_dataset=low_call_voices['test'],  # 평가 데이터셋
    compute_metrics=compute_metrics,  # 평가 메트릭 계산 함수
    tokenizer=small_processor,  # processor로 변경
    data_collator= small_data_collator # 데이터 패딩,
)


# tiny 모델 평가
tiny_eval_result = tiny_trainer.evaluate()
print(tiny_eval_result)

{'eval_loss': 5.8327717781066895, 'eval_cer': 276.53589135589567, 'eval_runtime': 177.0691, 'eval_samples_per_second': 4.015, 'eval_steps_per_second': 0.503}


In [ ]:
from transformers import DataCollatorForSeq2Seq

base_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")
base_processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="Korean", task="transcribe")
base_tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", language="Korean", task="transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")

base_data_collator = DataCollatorForSeq2Seq(
    tokenizer=base_processor.feature_extractor,  # fine_processor.feature_extractor 대신 tokenizer 사용
    model=base_model,  # fine_model 대신 base_model 사용
    padding='longest',
    pad_to_multiple_of=None
)

In [ ]:
# base 모델
from transformers import WhisperTokenizer,  WhisperFeatureExtractor
from transformers import WhisperProcessor
from transformers import Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="repo_name",  # 원하는 리포지토리 이름을 임력한다.
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # 배치 크기가 2배 감소할 때마다 2배씩 증가
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="cer",  # 한국어의 경우 'wer'보다는 'cer'이 더 적합할 것
    greater_is_better=False,
    push_to_hub=False,
)

base_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")
base_trainer = Seq2SeqTrainer(
    model=base_model,
    args=training_args,
    eval_dataset=low_call_voices['test'],  # 평가 데이터셋
    compute_metrics=compute_metrics,  # 평가 메트릭 계산 함수
    tokenizer=base_processor,  # processor로 변경
    data_collator= base_data_collator # 데이터 패딩,
)


# base 모델 평가
base_eval_result = base_trainer.evaluate()
print(base_eval_result)

{'eval_loss': 3.794567108154297, 'eval_cer': 211.8991161888338, 'eval_runtime': 187.8978, 'eval_samples_per_second': 3.784, 'eval_steps_per_second': 0.474}


In [ ]:
from transformers import DataCollatorForSeq2Seq

medium_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium")
medium_processor = WhisperProcessor.from_pretrained("openai/whisper-medium", language="Korean", task="transcribe")
medium_tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-medium", language="Korean", task="transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-medium")

medium_data_collator = DataCollatorForSeq2Seq(
    tokenizer=medium_processor.feature_extractor,
    model=medium_model,  # fine_model 대신 medium_model 사용
    padding='longest',
    pad_to_multiple_of=None
)

In [ ]:
# medium 모델
from transformers import WhisperTokenizer,  WhisperFeatureExtractor
from transformers import WhisperProcessor
from transformers import Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="repo_name",  # 원하는 리포지토리 이름을 임력한다.
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # 배치 크기가 2배 감소할 때마다 2배씩 증가
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="cer",  # 한국어의 경우 'wer'보다는 'cer'이 더 적합할 것
    greater_is_better=False,
    push_to_hub=False,
)

medium_trainer = Seq2SeqTrainer(
    model=medium_model,
    args=training_args,
    eval_dataset=low_call_voices['test'],  # 평가 데이터셋
    compute_metrics=compute_metrics,  # 평가 메트릭 계산 함수
    tokenizer=medium_processor,
    data_collator= medium_data_collator # 데이터 패딩,
)


# base 모델 평가
medium_eval_result = medium_trainer.evaluate()
print(medium_eval_result)

{'eval_loss': 4.498866081237793, 'eval_cer': 185.7943522310843, 'eval_runtime': 621.3612, 'eval_samples_per_second': 1.144, 'eval_steps_per_second': 0.143}


In [ ]:
from transformers import WhisperTokenizer,  WhisperFeatureExtractor
from transformers import WhisperProcessor
from transformers import Seq2SeqTrainer
from transformers import WhisperForConditionalGeneration
from transformers import Seq2SeqTrainer

In [ ]:
fine_model = WhisperForConditionalGeneration.from_pretrained("aoome123/repo_name")
feature_extractor = WhisperFeatureExtractor.from_pretrained("aoome123/repo_name")
tokenizer = WhisperTokenizer.from_pretrained("aoome123/repo_name", language="Korean", task="transcribe")
fine_processor = WhisperProcessor.from_pretrained("aoome123/repo_name")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import WhisperForConditionalGeneration
from transformers import DataCollatorForSeq2Seq

fine_data_collator = DataCollatorForSeq2Seq(
    tokenizer=fine_processor.feature_extractor,  # fine_processor.feature_extractor 대신 tokenizer 사용
    model=fine_model,  # fine_model 대신 base_model 사용
    padding='longest',
    pad_to_multiple_of=None
)

In [ ]:
# fine-tuning 모델
from transformers import WhisperTokenizer,  WhisperFeatureExtractor
from transformers import WhisperProcessor
from transformers import Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="repo_name",  # 원하는 리포지토리 이름을 임력한다.
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # 배치 크기가 2배 감소할 때마다 2배씩 증가
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="cer",  # 한국어의 경우 'wer'보다는 'cer'이 더 적합할 것
    greater_is_better=False,
    push_to_hub=False,
)

fine_model = WhisperForConditionalGeneration.from_pretrained("aoome123/repo_name")
fine_trainer = Seq2SeqTrainer(
    model=fine_model,
    args=training_args,
    eval_dataset=low_call_voices['test'],  # 평가 데이터셋
    compute_metrics=compute_metrics,  # 평가 메트릭 계산 함수
    tokenizer=fine_processor,
    data_collator= fine_data_collator # 데이터 패딩,
)


# fine_tuning 모델 평가
fine_eval_result = fine_trainer.evaluate()
print(fine_eval_result)

{'eval_loss': 0.19245672225952148, 'eval_cer': 9.560250053890924, 'eval_runtime': 177.1212, 'eval_samples_per_second': 4.014, 'eval_steps_per_second': 0.502}
